# Very naive learning on the vector of surrounding bases with standardish machine learning algorithms from scikit sklearn

---

### Analysis

I will try to naively predict the mehtylation value (0,1) based on naive sequence context feauture vectors.

I will use a very popular "state of the art" general machine learning library: xgboost.
    



### Conlcusions

- It gains a slight, but real increse in precision over the other standard models (3-4%)

---

In [1]:
import numpy as np

import time
import xgboost as xgb

import sys
sys.path.append('../my_modules')
from loading_utils import read_my_data,create_sets

#workdir='/nagyvinyok/adat84/sotejedlik/ribli/methylation_code/modelling'
#subprocess.call(['mkdir',workdir])
#os.chdir(workdir)

Couldn't import dot_parser, loading of dot files will not be possible.


### Load data

In [2]:
x,y= read_my_data(fname='../prepare_data/small_meth_feat_vect.csv')

#select the whole dataset
(train_x,train_y),(valid_x,valid_y),(test_x,test_y)= create_sets(
    x,y,N_train=80000,N_valid=10000,N_test=10000,length=2000)

Loading data... 


#### Create xgboost matrices

In [3]:
dtrain = xgb.DMatrix( train_x, label=train_y)
dvalid = xgb.DMatrix( valid_x, label=valid_y)
dtest = xgb.DMatrix( test_x, label=test_y)

#### Define params

In [12]:
param = {'bst:max_depth':7,
         'bst:eta':0.01,
         'min_child_weight':1,
         'colsample_bytree':1,
         'silent':1,
         'objective': "binary:logistic",
         'eval_metric': 'error',
         'nthread':4}

evallist  = [(dvalid,'eval'), (dtrain,'train')]

#### Fit

- The evaluation set is used to monitor the learning

In [13]:
#train
num_round = 2000
bst = xgb.train(param,
                dtrain,
                evals=evallist,
                num_boost_round=num_round)
                #early_stopping_rounds=5,
                #feval=feval) #not using my own evaluation function because its sloW!

train_bst_pred=bst.predict(dtrain)
test_bst_pred=bst.predict(dvalid)

[0]	eval-error:0.381500	train-error:0.377613
[1]	eval-error:0.380600	train-error:0.378350
[2]	eval-error:0.377400	train-error:0.366087
[3]	eval-error:0.375400	train-error:0.366237
[4]	eval-error:0.374400	train-error:0.363037
[5]	eval-error:0.376900	train-error:0.362262
[6]	eval-error:0.363100	train-error:0.351600
[7]	eval-error:0.357100	train-error:0.344712
[8]	eval-error:0.353000	train-error:0.338237
[9]	eval-error:0.346900	train-error:0.328788
[10]	eval-error:0.345800	train-error:0.327875
[11]	eval-error:0.341300	train-error:0.321012
[12]	eval-error:0.340100	train-error:0.318325
[13]	eval-error:0.331100	train-error:0.310387
[14]	eval-error:0.327200	train-error:0.304275
[15]	eval-error:0.322600	train-error:0.298875
[16]	eval-error:0.317500	train-error:0.292425
[17]	eval-error:0.318200	train-error:0.293175
[18]	eval-error:0.316100	train-error:0.289500
[19]	eval-error:0.314400	train-error:0.284662
[20]	eval-error:0.312500	train-error:0.281350
[21]	eval-error:0.310200	train-error:0.27947

#### Final scores

In [14]:
print 'train score:',list(map(round,bst.predict(dtrain))==train_y).count(True)/float(len(train_y))
print 'validation score:',list(map(round,bst.predict(dvalid))==valid_y).count(True)/float(len(valid_y))
print 'test score:',list(map(round,bst.predict(dtest))==test_y).count(True)/float(len(test_y))

train score: 0.973
validation score: 0.7919
test score: 0.7966
